## Database setup

In [21]:
import json
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base

In [2]:
# Define the database connection string
DATABASE_URL = 'sqlite:///challenge.db'  # You can replace this with your desired database URL

# Create an SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Define the base class for declarative models
Base = declarative_base()

In [3]:
# Define the Product table
class Product(Base):
    __tablename__ = 'Product'
    Id = Column(Integer, primary_key=True)
    Title = Column(String(100))

In [4]:
# Define the ProductDescription table
class ProductDescription(Base):
    __tablename__ = 'ProductDescription'
    Id = Column(Integer, primary_key=True)
    ProductId = Column(Integer, ForeignKey('Product.Id'))
    CountryCode = Column(Text)
    OriginalText = Column(Text)
    TranslatedText = Column(Text)
    product = relationship("Product")

In [5]:
# Define the ProductImages table
class ProductImages(Base):
    __tablename__ = 'ProductImages'
    Id = Column(Integer, primary_key=True)
    ImageId = Column(Integer, ForeignKey('Image.Id'))
    ProductId = Column(Integer, ForeignKey('Product.Id'))
    ImageIndex = Column(Integer)
    product = relationship("Product")
    image = relationship("Image")

In [6]:
# Define the Image table
class Image(Base):
    __tablename__ = 'Image'
    Id = Column(Integer, primary_key=True)
    Url = Column(Text)

---

In [7]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [8]:
# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
# Populate with sample data
product1 = Product(Title="Product 1")
product2 = Product(Title="Product 2")

In [10]:
desc1 = ProductDescription(
    ProductId=1, CountryCode="US",
    OriginalText="Original text 1",
    TranslatedText="Translated text 1"
)
desc2 = ProductDescription(
    ProductId=1, CountryCode="UK",
    OriginalText="Original text 2",
    TranslatedText="Translated text 2"
)

In [11]:
image1 = Image(Url="https://example.com/image1.jpg")
image2 = Image(Url="https://example.com/image2.jpg")

In [12]:
product_image1 = ProductImages(ProductId=1, ImageId=1, ImageIndex=0)
product_image2 = ProductImages(ProductId=1, ImageId=2, ImageIndex=0)
product_image3 = ProductImages(ProductId=2, ImageId=2, ImageIndex=1)

In [13]:
# Add data to the session
session.add_all([
    product1, product2,
    desc1, desc2,
    image1, image2,
    product_image1, product_image2, product_image3
])

# Commit the changes to the database
session.commit()

------

## Challenge Query

Extract, for every existing product, the following fields:
- `Product.Title`
- `Image.Url` for the images with the ImageIndex = 0. *ImageIndex field states the priority order of images of a certain product. So for a given ProductId, the image with ImageIndex = 0 would be the most relevant image for that product.*
- `ProductDescription.TranslatedText` if exists, else `ProductDescription.OriginalText` for `ProductDescriptions` in `CountryCode = ‘us’`.

In [14]:
all_products = session.query(Product).all()

In [19]:
res = []
for product in all_products:
    data = {"title": product.Title}
    data["descriptions"] = []
    data["images"] = []
    
    product_images = session.query(ProductImages).filter_by(ProductId=product.Id, ImageIndex=0).all()
    for product_image in product_images:
        image = session.query(Image).filter_by(Id=product_image.ImageId).first()
        data["images"].append(image.Url)

    product_descriptions = session.query(ProductDescription).filter_by(ProductId=product.Id).all()
    for description in product_descriptions:
        cc = description.CountryCode
        if description.TranslatedText:
            data["descriptions"].append({
                "cc": cc, "type": "translated", "text": description.TranslatedText
            })
        elif description.OriginalText and cc.lower() == "us":
            data["descriptions"].append({
                "cc": "us", "type": "original", "text": description.OriginalText
            })

    res.append(data)

In [22]:
print(json.dumps(res, sort_keys=True, indent=4))

[
    {
        "descriptions": [
            {
                "cc": "US",
                "text": "Translated text 1",
                "type": "translated"
            },
            {
                "cc": "UK",
                "text": "Translated text 2",
                "type": "translated"
            }
        ],
        "images": [
            "https://example.com/image1.jpg",
            "https://example.com/image2.jpg"
        ],
        "title": "Product 1"
    },
    {
        "descriptions": [],
        "images": [],
        "title": "Product 2"
    }
]
